In [1]:
import time
import math
import numpy as np
import numba as nb
from numba import cuda
import torch

import warnings
warnings.filterwarnings('ignore')

In [2]:
nb.cuda.detect()

Found 1 CUDA devices
id 0    b'NVIDIA GeForce RTX 2070 SUPER'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-d76dbc1d-cbf5-d7ba-7ff2-b858d30bc8f2
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [3]:
sizes = [256, 1024, 4096]
iterations = 10

## numba (CUDA)　での行列積速度

1回目は速度が安定しないので、2回実行して2回目の結果を利用

In [31]:
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 32

In [32]:
@nb.cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = nb.cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [33]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]))

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations

    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy*1000}")
    print(f"処理時間 : kernel = {d_sec_kernel*1000}")
    print(f"処理時間 : total  = {d_sec_total*1000}")

行列サイズ = 256
処理時間 : memcpy = 0.7894992828369141
処理時間 : kernel = 0.7791042327880859
処理時間 : total  = 1.6638755798339844
行列サイズ = 1024
処理時間 : memcpy = 2.187681198120117
処理時間 : kernel = 31.35361671447754
処理時間 : total  = 34.05449390411377
行列サイズ = 4096
処理時間 : memcpy = 21.334099769592285
処理時間 : kernel = 1790.723180770874
処理時間 : total  = 1822.3876237869263


In [34]:
@nb.cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)
    sB = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)

    x, y = nb.cuda.grid(2)

    tx = nb.cuda.threadIdx.x
    ty = nb.cuda.threadIdx.y
    bpg = nb.cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        nb.cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        nb.cuda.syncthreads()

    C[x, y] = tmp

In [35]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]))

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        fast_matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy*1000}")
    print(f"処理時間 : kernel = {d_sec_kernel*1000}")
    print(f"処理時間 : total  = {d_sec_total*1000}")

行列サイズ = 256
処理時間 : memcpy = 0.7711172103881836
処理時間 : kernel = 0.865483283996582
処理時間 : total  = 1.732468605041504
行列サイズ = 1024
処理時間 : memcpy = 2.1803855895996094
処理時間 : kernel = 35.29694080352783
処理時間 : total  = 37.98873424530029
行列サイズ = 4096
処理時間 : memcpy = 21.339106559753418
処理時間 : kernel = 2011.0787153244019
処理時間 : total  = 2042.753553390503


## numba ブロックあたりのスレッド数を変えた場合

In [36]:
TPB = 16

In [37]:
@nb.cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = nb.cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [38]:
@nb.cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)
    sB = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)

    x, y = nb.cuda.grid(2)

    tx = nb.cuda.threadIdx.x
    ty = nb.cuda.threadIdx.y
    bpg = nb.cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        nb.cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        nb.cuda.syncthreads()

    C[x, y] = tmp

In [39]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB, 1)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]), 1)

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy*1000}")
    print(f"処理時間 : kernel = {d_sec_kernel*1000}")
    print(f"処理時間 : total  = {d_sec_total*1000}")

行列サイズ = 256
処理時間 : memcpy = 0.771331787109375
処理時間 : kernel = 0.423431396484375
処理時間 : total  = 1.2894630432128906
行列サイズ = 1024
処理時間 : memcpy = 2.1795034408569336
処理時間 : kernel = 17.740178108215332
処理時間 : total  = 20.432591438293457
行列サイズ = 4096
処理時間 : memcpy = 21.906471252441406
処理時間 : kernel = 994.7253704071045
処理時間 : total  = 1026.9135236740112


In [45]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]))

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        fast_matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy*1000}")
    print(f"処理時間 : kernel = {d_sec_kernel*1000}")
    print(f"処理時間 : total  = {d_sec_total*1000}")

行列サイズ = 256
処理時間 : memcpy = 1.0704278945922852
処理時間 : kernel = 0.4372119903564453
処理時間 : total  = 1.6028881072998047
行列サイズ = 1024
処理時間 : memcpy = 2.2068023681640625
処理時間 : kernel = 17.43490695953369
処理時間 : total  = 20.154881477355957
行列サイズ = 4096
処理時間 : memcpy = 21.332454681396484
処理時間 : kernel = 993.3174848556519
処理時間 : total  = 1024.927306175232


## pytorchでの行列積速度

In [58]:
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

device = torch.device(0)

In [59]:
for size in sizes:
    x = torch.randn((size, size), dtype=torch.float32, requires_grad=False)
    y = torch.randn((size, size), dtype=torch.float32, requires_grad=False)

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        with torch.no_grad():
            start = time.time()

            d_x = x.to(device)
            d_y = y.to(device)

            # 時間計測 (memcpy)
            torch.cuda.synchronize()
            memcpy_end = time.time()
            if i != 0:
                d_sec_memcpy += (memcpy_end - start)


            d_out = torch.matmul(d_x, d_y)

            # 時間計測 (kernel)
            torch.cuda.synchronize()
            kernel_end = time.time()
            if i != 0:
                d_sec_kernel += (kernel_end - memcpy_end)


            out = d_out.to("cpu")

            # 時間計測 (total)
            torch.cuda.synchronize()
            end = time.time()
            if i != 0:
                d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy*1000}")
    print(f"処理時間 : kernel = {d_sec_kernel*1000}")
    print(f"処理時間 : total  = {d_sec_total*1000}")

行列サイズ = 256
処理時間 : memcpy = 0.09441375732421875
処理時間 : kernel = 0.03135204315185547
処理時間 : total  = 0.1891613006591797
行列サイズ = 1024
処理時間 : memcpy = 0.8060216903686523
処理時間 : kernel = 0.3333568572998047
処理時間 : total  = 1.6060590744018555
行列サイズ = 4096
処理時間 : memcpy = 11.906719207763672
処理時間 : kernel = 15.307140350341797
処理時間 : total  = 46.6691255569458


In [60]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [61]:
for size in sizes:
    x = torch.randn((size, size), dtype=torch.float32, requires_grad=False)
    y = torch.randn((size, size), dtype=torch.float32, requires_grad=False)

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        with torch.no_grad():
            start = time.time()

            d_x = x.to(device)
            d_y = y.to(device)

            # 時間計測 (memcpy)
            torch.cuda.synchronize()
            memcpy_end = time.time()
            if i != 0:
                d_sec_memcpy += (memcpy_end - start)


            d_out = torch.matmul(d_x, d_y)

            # 時間計測 (kernel)
            torch.cuda.synchronize()
            kernel_end = time.time()
            if i != 0:
                d_sec_kernel += (kernel_end - memcpy_end)


            out = d_out.to("cpu")

            # 時間計測 (total)
            torch.cuda.synchronize()
            end = time.time()
            if i != 0:
                d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy*1000}")
    print(f"処理時間 : kernel = {d_sec_kernel*1000}")
    print(f"処理時間 : total  = {d_sec_total*1000}")

行列サイズ = 256
処理時間 : memcpy = 0.09737014770507812
処理時間 : kernel = 0.030660629272460938
処理時間 : total  = 0.18973350524902344
行列サイズ = 1024
処理時間 : memcpy = 0.7985830307006836
処理時間 : kernel = 0.33249855041503906
処理時間 : total  = 1.5969514846801758
行列サイズ = 4096
処理時間 : memcpy = 11.87746524810791
処理時間 : kernel = 16.003704071044922
処理時間 : total  = 47.33924865722656
